<img src="images/quantum_algorithms_tutorials.png" alt="drawing" width="100" align="left"/>

<h1 align="center">
	Quantum phase estimation
</h1>

The quantum phase estimation (QPE) algorithm is a quantum algorithm for estimating the phase of an eigenvector of a unitary operator [1]. One of the main applications of the algorithm is to estimate eigenvalues (energies) of some molecule's $H$ Hamiltonian [2]. Phase estimation is frequently used as a subroutine in other quantum algorithms, such as Shor's algorithm and the quantum algorithm for linear systems of equations [1]. 

The goal of the algorithm is to estimate the eigenvalue of a given Hamiltonian $\hat{H}$. We exponantiate it and obtain $U = e^{i\hat{H}t}$ unitary operator, where $t$ is some parameter (in our example $t = 1$). Then we apply quantum circuit that corresponds to $U = e^{i\hat{H}t}$ on some eigenvector of the $\hat{H}$. For simplicity we will take such $\hat{H}$, which eigenvectors will be the states $\left| 0 \right\rangle$ and $\left| 1 \right\rangle$ (for this $\hat{H}$ should be a diaganal matrix). In our example we will try to find eigenvalue that corresponds to $\left| 1 \right\rangle$ eigenvector. More mathematically:

\begin{align}
\hat{H} = \begin{pmatrix}
E_1 & 0\\
0 & E_2
\end{pmatrix} \longrightarrow
e^{i\hat{H}t} = \begin{pmatrix}
e^{i E_1 t} & 0\\
0 & e^{i E_2 t} 
\end{pmatrix} \longrightarrow e^{i H t} \left| 1 \right\rangle = e^{i E_2 t} \left| 1 \right\rangle = e^{i 2 \pi \varphi_2} \left| 1 \right\rangle,
\end{align}

where $E_1$ and $E_2$ are the eigenvalues of the $\hat{H}$, $\varphi_2 = \frac{E_2 t}{2 \pi}$ is the phase that the algorithm is capable to estimate. Note that $U = e^{i H t}$ is a unitary operator and in code we will name it as ```unitary```.

The code and the algorithm is similar to the [IPEA](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/iterative_phase_estimation.ipynb) algorithm, which tutorial we also have covered. IPEA actually is a modification of QPE algorithm. So, that's way some parts/explanations are omited here and more details are proveded in the parts of QPE that is different from IPEA.

In the first code, all nesassary packages are imported and the diaganal matrix is created:

In [1]:
import numpy as np
from random import random
from qiskit import *
from qiskit.aqua.operators import WeightedPauliOperator, MatrixOperator
from qiskit.aqua.operators.op_converter import to_matrix_operator
from qiskit.aqua.utils.controlled_circuit import get_controlled_circuit
from qiskit.extensions.standard.barrier import Barrier

def simple_hamiltonian_operator(E_1, E_2):
    """
    Creates 0.5*(E_1 + E_2)*I + 0.5*(E_1 - E_2)*Z pauli sum 
    that will be our "simple" Hamiltonian operator. The corresponding 
    matrix for Hamiltonian is [[E_1, 0], [0, E_2]].
    
    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": 0.5 * (E_1 - E_2)}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": 0.5 * (E_1 + E_2)}, "label": "I"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

E_1, E_2 = (0, 2*np.pi*random())
print("We are going to estimate E_2 with QPE algorithm = {}".format(E_2))

H = simple_hamiltonian_operator(E_1, E_2)
print("\nThe Hamiltonian operator in the matrix form:")
print(to_matrix_operator(H).dense_matrix)


We are going to estimate E_2 with QPE algorithm = 3.6599951224912908

The Hamiltonian operator in the matrix form:
[[0.        +0.j 0.        +0.j]
 [0.        +0.j 3.65999512+0.j]]


In the folowing code the quantum ```circuit``` is created with 
* $q$ quantum state register, where we encode the $\left| 1 \right\rangle$ eigenvector by appling ```x``` gate.

* $a$ ancllary quantum reqister, where in the course of the algorithm the bits of the phase $\varphi_2$ will be stored.

* $c$ classical register, where the measurement results will be stored.

In [2]:
num_bits_estimate = 5
# For 2x2 matrix one qubit is enough
q = QuantumRegister(1, name="q")
# In QPE we use n ancillas to estimate n bits from the phase
a = QuantumRegister(num_bits_estimate, name="a") 
# For n ancillary qubit measurment we need n cllasical bits
c = ClassicalRegister(num_bits_estimate, name="c") 

# Create a quantum circuit
circuit = QuantumCircuit(q, a, c)

# |1> eigenstate initialization
circuit.x(q[0])

Here we obtain a circuit for $U = e^{iHt}$. 

In [3]:
t = 1
unitary = H.evolve(evo_time=-t, quantum_registers=q)
# In some versions of qiskit aqua evolve adds a Barrier
# at the end of the circuit, that causes errors when we
# use get_controlled_circuit method in older versions. 
# Here we check is there a Barrier. If so, we will delete it.
if isinstance(unitary._data[-1][0], Barrier):
    unitary.data.__delitem__(-1)

Firstly we apply Hadamard gates to all ancillary qubits:

In [4]:
for ancillary in a:
    circuit.h(ancillary)

The quantum state of ancillary qubits before Hadamard gates:
\begin{align*}
\left| \psi_a \right\rangle = \left| 00000 \right\rangle = \left| 0 \right\rangle \left| 0 \right\rangle \left| 0 \right\rangle \left| 0 \right\rangle \left| 0 \right\rangle
\end{align*}

The quantum state of ancillary qubits after Hadamard gates:
\begin{align*}
&\left| \psi_a \right\rangle = \frac{1}{2^{5/2}}
\left(\left| 0 \right\rangle + \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + \left| 1 \right\rangle \right) = \\
&\left| +++++ \right\rangle = \left| + \right\rangle \left| + \right\rangle \left| + \right\rangle \left| + \right\rangle \left| + \right\rangle,
\end{align*}

because $H \left| 0 \right\rangle = \frac{1}{\sqrt{2}}
\left(\left| 0 \right\rangle + \left| 1 \right\rangle \right) = \left| + \right\rangle$. Please don't confuse Hadamard gate $H$ with Hermitian operator $\hat{H}$, because we use the same letter for both of them.

For general case we will represent $\varphi$ in its fractional binary representation:

$$\varphi = 0.\varphi_0 \varphi_1 \varphi_2 \varphi_3... =  \frac{\varphi_0}{2} + \frac{\varphi_1}{2^2} + \frac{\varphi_2}{2^3} + \frac{\varphi_3}{2^4} + ...,$$

where each $\varphi_k$ is a bit (it is equal to $0$ or $1$).

Now we apply from the controlled version of circuit for $U = e^{i \hat{H} t}$ (```unitary```) to $q$ quantum register controlled by first ancillary qubit. This will give us:

$$\left|+ \right\rangle_a \left|1 \right\rangle_q = \frac{1}{\sqrt{2}} (\left|0 \right\rangle_a + \left|1 \right\rangle_a)\left|1 \right\rangle_q 
\xrightarrow{e^{i \hat{H} t}}
\left|0 \right\rangle_{a_0} \left|1 \right\rangle_q + \left|1 \right\rangle_{a_0} e^{-i 2 \pi \varphi} \left|1 \right\rangle_q = 
\left( \left|0 \right\rangle_{a_0} + e^{-i 2 \pi \varphi} \left|1 \right\rangle_{a_0}\right)\left|1 \right\rangle_q$$

This way we will have the phase in the phase of the first ancillary qubit. Also, we apply controled version of ```unitary``` $2^{n-1}$ times in seqsetion to $q$ register controlled by $n$th ancillary qubit ($n$ starts from $0$ to $4$). After this the state of ancillary qubits will look like this:

\begin{align*}
\left| \psi_a \right\rangle = \frac{1}{2^{5/2}}
\left(\left| 0 \right\rangle + e^{i 2 \pi \varphi} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 2\varphi} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 4\varphi} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 8\varphi} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 16\varphi} \left| 1 \right\rangle \right).
\end{align*}

By taking into accout the fractional binary representation of $\varphi$:

\begin{align*}
\left| \psi_a \right\rangle = \frac{1}{2^{5/2}}
&\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_0 \varphi_1 \varphi_2 \varphi_3 \varphi_4} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_1 \varphi_2 \varphi_3 \varphi_4} \left| 1 \right\rangle \right) \\
&\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_2 \varphi_3 \varphi_4} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_3 \varphi_4} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_4} \left| 1 \right\rangle \right)
\end{align*}.

In the code we should apply  $2^{n}$ times controled version of ```unitary``` on $q$ register controlled by $n$th qubit ($n \in (0, n)$). Here is the method for this part:

In [5]:
for n in range(a.size):
    for m in range(n):
        get_controlled_circuit(unitary, a[n], circuit)

After runining this method we will obtain the following circuit:

<img src="images/pea_control_5ancillary.png" alt="drawing" width="300"/>

At the next step we will apply a Hadamard gate to the $n = 5$ qubit and controlled rotations $R_z(\frac{1}{2^{n - m + 1}})$ from $n$th qubit to the rest $m < n$ qubits. This will give us a quantum state: 

\begin{align*}
\left| \psi_a \right\rangle = \frac{1}{2^{5/2}}
&\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_0 \varphi_1 \varphi_2 \varphi_3} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_1 \varphi_2 \varphi_3} \left| 1 \right\rangle \right) \\
&\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_2 \varphi_3} \left| 1 \right\rangle \right)
\left(\left| 0 \right\rangle + e^{i 2 \pi 0.\varphi_3} \left| 1 \right\rangle \right)
\left| \varphi_4 \right\rangle.
\end{align*}

Here we take into accout that when $\varphi_4 = 0$ the state of the $5$th qubit is equal to $\left| 0 \right\rangle + \left| 1 \right\rangle = \left| + \right\rangle$ and when $\varphi_4 = 1$ it equal to $\left| 0 \right\rangle - \left| 1 \right\rangle = \left| - \right\rangle$. Also

\begin{align*}
H \left| + \right\rangle= \left| 0 \right\rangle \qquad
\qquad
H \left| - \right\rangle = \left| 1 \right\rangle
\end{align*}

We repeat this procidure for all qubits and at the end the quantum state will look like: 

\begin{align*}
\left| \psi_a \right\rangle=
\left| \varphi_0 \right\rangle
\left| \varphi_1 \right\rangle
\left| \varphi_2 \right\rangle
\left| \varphi_3 \right\rangle
\left| \varphi_4 \right\rangle.
\end{align*}

So, by measuring each qubit we will obtain all bits of the phase. This part when we apply Hadamard gates and controll rotations is well known procedue called inverse Quauntum Fourie Transformation (inverse QFT) [3] without SWAP gates. Here is the code that adds this part of the algorithm:

In [6]:
# inverse QFT without SWAP gates
for n in reversed(range(a.size)):
    circuit.h(a[n])
    if n != 0:
        for m in reversed(range(n)):
            angle = -2*np.pi / (2**(n - m + 1))
            circuit.crz(angle, a[n], a[m])

# measurements on the ancillary qubits stored in c classical register
for n in reversed(range(a.size)):
    circuit.measure(a[n],c[n])

The quantum circuit that corresponds to the whole algorithm looks like this (here we draw a circuit that has only 4 ancillary qubits, because for 5 ancillary qubits the image is much bigger):

<img src="images/pea_circuit_4ancillary.png" alt="drawing" width="800"/>

At the final piece of code we run the algorithm on QC simulator and add measurement results stored in $c$ classical register that gives us the estimated value for the phase. After it, by remambering that $E_2 = \frac{2 \pi \varphi_2}{t}$, we obtain desired $E_2$ eigenvalue of the $\hat{H}$ Hamiltonian:

In [7]:
backend = BasicAer.get_backend('qasm_simulator')
shots = 2048  # how many time execute the algorithm
job = execute(circuit, backend, shots=shots)
result = job.result()
counts = result.get_counts()

phase_bits = max(counts, key=counts.get) # take the most often obtaned result

phase = 0
for index, bit in enumerate(phase_bits):
    phase += int(bit) / 2**(index + 1)
    
estimated_E_2 = 2 * np.pi * phase / t

print("Eigenvalue of Hamiltonian: {}".format(E_2))
print("Estimated eigenvalue of Hamiltonian: {}".format(estimated_E_2))

Eigenvalue of Hamiltonian: 3.6599951224912908
Estimated eigenvalue of Hamiltonian: 5.301437602932776


**Final notes**: Here we didn't implemented classical textbook version of QPE algorithm. In textbooks [4] the algorithm is done via inverse QFT with SWAP gates. If we want to implement the algorithm with inverse QFT, we should change a little. Firstly the order of control ```unitary``` gates should be changed: $0$th ancllary should controll $U^n$ gate on $q$, instead of $U$ gate, register and the last $n$th qubit should control $U$ gate, instead of $U^n$. Secondly SWAP gates are added after control unitary part of the algorithm in order to obtain invers QFT. Here is the textbook [4] implementation of the algorithm:

<img src="images/textbook_pea.png" alt="drawing" width="800"/>

where the inverse QFT is the following part of the circuit:

<img src="images/inverse_QFT.png" alt="drawing" width="800"/>

How you have seen we did the job of the algorithm without SWAP gates and obtained the result. The SWAP gates doesn't add anything to this algorithm and we can omit them. Moreover SWAP gates are two qubit gates that in real quantum computer in general have big error rate. Not only they increase the gate number of the circuit, but also they add errors. So, this is the motivation that we avoided to use the SWAP gates.

Also they are some details about Trotterization, the $t$ parameter in the $U = e^{iHt}$, what should we do when the eigenvalue is negative that we don't cover here. If you interested in this topics you can check out our [IPEA](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/iterative_phase_estimation.ipynb) tutorial, that is a modification of QPE algorithm. There are two main differences between QPE and IPEA algorithms. QPE demands $n$ ancillary qubits for $n$ bit phase estimation, IPEA demands only one ancillary qubit for any $n$ bit phase estimation. The second difference is that QPE applies inverse QFT, and IPEA applies inverse QFT iteratively ($R_z$ rotations are applied iteratively).

[1] [Quantum phase estimation algorithm (wiki)](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm)

[2] [Alán Aspuru-Guzik et al., Science 309 , 1704 (2005)](https://science.sciencemag.org/content/309/5741/1704)

[3] [Quantum Fourier transform (wiki)](https://en.wikipedia.org/wiki/Quantum_Fourier_transform)

[4] [M.A. Nielsen, I.L. Chuang, Cambridge University Press New York, "Quantum Computation and Quantum Information: 10th Anniversary Edition
10th" (2011)](https://www.cambridge.org/am/academic/subjects/physics/quantum-physics-quantum-information-and-quantum-computation/quantum-computation-and-quantum-information-10th-anniversary-edition?format=HB)